In [122]:
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

corpus de entrenamiento

In [123]:
# Cargar el corpus y las respuestas desde el CSV
df = pd.read_csv("chatbot_corpus_respuestas.csv")

corpus = df["entrada"].tolist()
respuestas = df["respuesta"].tolist()

obtiene los embeddings del corpus

In [124]:
# 1. Vectorizador
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=1000,               # Número máximo de palabras únicas en el vocabulario, aumentar si el cospus crece
    output_mode='int',             # Salida como secuencia de enteros (índices de palabras)
    output_sequence_length=100       # Longitud fija de la secuencia de salida, aumentar si las frases son mas largas
)
vectorizer.adapt(corpus)

# 2. Embedding
embedding_dim = 100 #aumentar si el dataset aumenta
embedding_layer = tf.keras.layers.Embedding(input_dim=1000, output_dim=embedding_dim)

# 3. Embeddings del corpus
corpus_tokens = vectorizer(corpus)
corpus_embeddings = embedding_layer(corpus_tokens)
corpus_vectors = tf.reduce_mean(corpus_embeddings, axis=1).numpy()

In [125]:
# 4. Función para responder
def responder_frase_top3(frase_usuario):
    token_usuario = vectorizer([frase_usuario])
    embedding_usuario = embedding_layer(token_usuario)
    vector_usuario = tf.reduce_mean(embedding_usuario, axis=1).numpy()

    # Calcular similitud coseno con cada frase del corpus
    similitudes = cosine_similarity(vector_usuario, corpus_vectors).flatten()

    # Obtener los 3 índices con mayor similitud
    top_3_indices = np.argsort(similitudes)[-3:][::-1]  # Orden descendente

    # Obtener las respuestas correspondientes
    top_3_respuestas = [respuestas[i] for i in top_3_indices]
    top_3_similitudes = [similitudes[i] for i in top_3_indices]

    return list(zip(top_3_respuestas, top_3_similitudes))

In [136]:
# 5. Probar con algunas entradas
entrada = "hasta mañana"

print(f"> Usuario: {entrada}")
respuestas_top3 = responder_frase_top3(entrada)

for i, (respuesta, score) in enumerate(respuestas_top3, start=1):
    print(f"{i}. 🤖 {respuesta} (similitud: {score:.10f})")

> Usuario: hasta mañana
1. 🤖 Espero verte pronto (similitud: 0.9998881221)
2. 🤖 Nos vemos pronto (similitud: 0.9998766780)
3. 🤖 Oh, entiendo (similitud: 0.9998711944)
